<a href="https://colab.research.google.com/github/wagnerhelio/ExtractPytonPDF/blob/main/Convercao_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importando Bibliotecas
!pip install tabula-py
!pip install PyPDF2

from PyPDF2 import PdfReader
import tabula
import pandas as pd
import csv
import re

# Expressão regular para identificar datas no formato DD/MM/YYYY seguidas de ponto final
date_pattern = re.compile(r'\b\d{2}/\d{2}/\d{4}\.')

# Abrir o arquivo PDF em modo de leitura binário
with open("dados.pdf", 'rb') as pdf_file:
    # Criar o leitor de PDF
    pdf_reader = PdfReader(pdf_file)

    # Obter o número de páginas do PDF
    number_of_pages = len(pdf_reader.pages)

    # Inicializar variáveis de controle
    start_line = 40
    found_start = False
    found_end = False
    full_text = ""

    # Iterar sobre as páginas do PDF
    for page_number in range(number_of_pages):
        page = pdf_reader.pages[page_number]
        # Extrair o texto da página
        text = page.extract_text()

        # Dividir o texto em linhas
        lines = text.splitlines()

        for i, line in enumerate(lines):
            if not found_start and i >= start_line:
                found_start = True

            if found_start and not found_end:
                if ("Goiânia, 05 de Novembro de 2024" in line) or ("Goiânia, 05 de Novembro de 2024" in line):
                    found_end = True
                    break
                full_text += line + " "  # Adicionar cada linha ao texto completo, separando com espaço

        if found_end:
            break

    # Dividir o texto completo nas posições onde o padrão de data seguido de ponto final é encontrado
    sentences = date_pattern.split(full_text)
    matched_dates = date_pattern.findall(full_text)

    # Nome do arquivo CSV
    csv_filename = "extracted_data.csv"

    # Exportar para CSV
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Escrever o cabeçalho
        writer.writerow(["Placa", "Numero_auto_infracao", "Data_infracao", "Codigo_Infracao", "Valor_Multa","Valor_Multa_Centavo","Data_Limite"])

        # Escrever as frases no CSV, removendo o ponto final das datas encontradas
        for i, sentence in enumerate(sentences):
            if sentence.strip():  # Ignorar sentenças vazias
                if i < len(matched_dates):
                    # Remover o ponto final da data encontrada e adicionar a frase
                    date_without_period = matched_dates[i].rstrip('.')
                    writer.writerow([sentence.strip() + " " + date_without_period])
                else:
                    writer.writerow([sentence.strip()])

print(f"Texto extraído exportado para o arquivo {csv_filename}")


Texto extraído exportado para o arquivo extracted_data.csv
